In [33]:
import random
import time
import sys
from collections import Counter
# from statistics import mean, median
# import tflearn
# from tflearn.layers.core import input_data, dropout, fully_connected
# from tflearn.layers.estimator import regression
import numpy as np
import grpc

# sys.path.append('./lib/proto/simulation')
import simulation_pb2_grpc
import simulation_pb2

In [38]:
with open('server-cert.pem', 'rb') as f:
    creds = grpc.ssl_channel_credentials(f.read())
channel = grpc.secure_channel('localhost:50051', creds)
stub = simulation_pb2_grpc.SimulationStub(channel)

# Helper Functions #

In [41]:
def randomAction():
    rand = random.randint(1, 3)
    if rand == 1:
        return "MOVE"
    if rand == 2:
        return "CONSUME"
    if rand == 3:
        return "WAIT"
    
def randomDirection():
    rand = random.randint(1, 4)
    if rand == 1:
        return "UP"
    if rand == 2:
        return "DOWN"
    if rand == 3:
        return "LEFT"
    if rand == 4:
        return "RIGHT"

# Test Commands #

In [39]:
stub.ResetWorld(simulation_pb2.Empty())

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Connect Failed"
	debug_error_string = "{"created":"@1551559447.562158000","description":"Failed to create subchannel","file":"src/core/ext/filters/client_channel/client_channel.cc","file_line":2721,"referenced_errors":[{"created":"@1551559447.562150000","description":"Pick Cancelled","file":"src/core/ext/filters/client_channel/lb_policy/pick_first/pick_first.cc","file_line":241,"referenced_errors":[{"created":"@1551559447.562133000","description":"Connect Failed","file":"src/core/ext/filters/client_channel/subchannel.cc","file_line":689,"grpc_status":14,"referenced_errors":[{"created":"@1551559447.562121000","description":"Failed to connect to remote host: OS Error","errno":61,"file":"src/core/lib/iomgr/tcp_client_posix.cc","file_line":205,"os_error":"Connection refused","syscall":"connect","target_address":"ipv4:127.0.0.1:50051"}]}]}]}"
>

In [30]:
agent = simulation_pb2.Agent(id="", x=1, y=2)

In [31]:
spawnAgentRes = stub.CreateAgent(simulation_pb2.CreateAgentRequest(agent=agent))
agentId = spawnAgentRes.id

In [32]:
obsv = stub.GetObservation(simulation_pb2.GetObservationRequest(agent=agentId))
cellList = list(obsv.cells)
print(cellList)

[u'EMPTY', u'EMPTY', u'EMPTY', u'EMPTY', u'EMPTY', u'FOOD', u'EMPTY', u'EMPTY']


In [5]:
cellList.insert(4, 'A')
cellListMatrix = np.array(cellList).reshape(3,3)
print(cellListMatrix)

[['EMPTY' 'FOOD' 'EMPTY']
 ['EMPTY' 'A' 'EMPTY']
 ['EMPTY' 'EMPTY' 'EMPTY']]


In [6]:
actionRes = stub.AgentAction(simulation_pb2.AgentActionRequest(Id=agentId, Direction="UP", Action="CONSUME"))
actionRes.Successful

True

In [23]:
obsv = stub.AgentObservation(simulation_pb2.AgentObservationRequest(Id=agentId))
cellList = list(obsv.Cells)
cellList.insert(4, 'A')
cellMatrix = np.array(cellList).reshape(3,3)
print(cellMatrix)

[['EMPTY' 'EMPTY' 'EMPTY']
 ['FOOD' 'A' 'EMPTY']
 ['EMPTY' 'EMPTY' 'EMPTY']]


In [16]:
print(obsv.Energy)

105


In [16]:
stub.ResetWorld(simulation_pb2.ResetWorldRequest())

In [ ]:
for cellUpdate in stub.Spectate(simulation_pb2.SpectateRequest(Id="test-id")):
    print(cellUpdate)

In [ ]:
stub.SubscribeToRegion(simulation_pb2.SubscribeToRegionRequest(Id="test-id", X=1, Y=1))

# Training #

In [15]:
initial_games = 2
goal_steps = 500
score_requirement = 32
training_data = []
scores = []
accepted_scores = []
for episode in range(initial_games):
    game_memory = []
    # Spawn a new agent
    spawnAgentRes = stub.SpawnAgent(simulation_pb2.SpawnAgentRequest(X=0, Y=0))
    agentId = spawnAgentRes.Id 
    print(agentId)
    # Score (essentially the age of the agent)
    score = 0
    # Perform actions and observations
    for t in range(goal_steps):
        obsv = stub.AgentObservation(simulation_pb2.AgentObservationRequest(Id=agentId))
        if obsv.Alive == False:
            break
        action = randomAction()
        direction = randomDirection()
        actionRes = stub.AgentAction(simulation_pb2.AgentActionRequest(Id=agentId, Direction=direction, Action=action))
        score += 1
        game_memory.append( [list(obsv.Cells), action] )
        time.sleep(.20)
    
    if score >= score_requirement:
        accepted_scores.append(score)
        for data in game_memory:
            training_data.append(data)
    
    # Reset the world
    stub.ResetWorld(simulation_pb2.ResetWorldRequest())
    
    scores.append(score)

# training_data_save = np.array(training_data)
# np.save('saved_training_data.npy', training_data_save)

# print('Average accepted score: ', mean(accepted_scores))
# print('Median accepted score : ', median(accepted_scores))
# print(Counter(accepted_scores))
        

_Rendezvous: <_Rendezvous of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "Method does not exist."
	debug_error_string = "{"created":"@1551205943.250572000","description":"Error received from peer","file":"src/core/lib/surface/call.cc","file_line":1017,"grpc_message":"Method does not exist.","grpc_status":5}"
>